# Wrangle data

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2
import os
from env import host, user, password, get_db_url
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import wrangle
import warnings
warnings.filterwarnings("ignore")

alpha = 0.5

## Acquire

In [ ]:
def get_zillow_data():
    """Seeks to read the cached zillow.csv first """
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        return get_new_zillow_data()



def get_new_zillow_data():
    '''this function gathers selected data from the ZILLOW SQL DF
    and uses the get_db_url function to connect to said dataframe'''
    sql = '''
    SELECT 
        bedroomcnt AS bedrooms, 
        bathroomcnt AS bathrooms,
        calculatedfinishedsquarefeet AS sq_ft,
        taxvaluedollarcnt AS tax_value,
        yearbuilt AS year_built,
        taxamount AS tax_amnt,
        lotsizesquarefeet AS lot_size,
        fips
    FROM
        properties_2017
       JOIN propertylandusetype using (propertylandusetypeid)
       JOIN predictions_2017 USING(parcelid)
    WHERE propertylandusedesc in ("Single Family Residential", 
                                  "Inferred Single Family Residential")
       AND transactiondate LIKE "2017%%";          
    '''
    return pd.read_sql(sql, get_db_url('zillow'))

In [ ]:
df = get_zillow_data()
df.head()

In [ ]:
df.shape

## Prepare

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum().sum() #/ len(df)

we will drop nulls as they are a about 1% of the total data and should not have an impacting effect on exploration or modeling

In [ ]:
df = df.dropna()

In [ ]:
df.duplicated().sum()
#minimal amount of duplicates so we will drop those too

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

In [ ]:
51835 / 52441

In [ ]:
1-0.9884441562899258

We still have over 98% of our data after drops

In [ ]:
df.fips.unique()

In [ ]:
list(df.columns)

In [ ]:
def mahalanobis(x=None, data=None, cov=None):

    x_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = np.linalg.inv(cov)
    left = np.dot(x_mu, inv_covmat)
    mahal = np.dot(left, x_mu.T)
    return mahal.diagonal()



In [ ]:
#create new column in dataframe that contains Mahalanobis distance for each row
df['mahalanobis'] = mahalanobis(x=df, data=df[['bedrooms',
 'bathrooms',
 'sq_ft',
 'tax_value',
 'year_built',
 'tax_amnt',
 'lot_size',
 'fips']])



In [ ]:
#display first five rows of dataframe
df.head()

In [ ]:
#calculate p-value for each mahalanobis distance 
df['p'] = 1 - chi2.cdf(df['mahalanobis'], 3)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df[df.p > 0.001].shape

In [ ]:
df.shape

In [ ]:
def remove_outliers(df):
    #create new column in dataframe that contains Mahalanobis distance for each row
    df['mahalanobis'] = mahalanobis(x=df, data=df[['bedrooms',
 'bathrooms',
 'sq_ft',
 'tax_value',
 'year_built',
 'tax_amnt',
 'lot_size',
 'fips']])
    
    #calculate p-value for each mahalanobis distance 
    df['p'] = 1 - chi2.cdf(df['mahalanobis'], 3)
    
    # drop rowss with p-value of less than 0.001
    df = df[df.p > 0.001]
    
    # drop calculative columns
    df = df.drop(['mahalanobis', 'p'], axis = 1)
    
    return df

In [ ]:
df = remove_outliers(df)
df.shape

In [ ]:
df.head()

In [ ]:
df.drop(['mahalanobis', 'p'], axis = 1)

In [2]:
df = wrangle.wrangle_zillow()


KeyError: "['taxamount'] not found in axis"

In [ ]:
df.head()

## Split

In [ ]:
def train_test_validate_split(df, test_size=.2, validate_size=.3, random_state=99):
    '''
    This function takes in a dataframe, then splits that dataframe into three separate samples
    called train, test, and validate, for use in machine learning modeling.
    Three dataframes are returned in the following order: train, test, validate. 
    
    The function also prints the size of each sample.
    '''
    train, test = train_test_split(df, test_size=.2, random_state=99)
    train, validate = train_test_split(train, test_size=.3, random_state=99)
    
    print(f'train\t n = {train.shape[0]}')
    print(f'test\t n = {test.shape[0]}')
    print(f'validate n = {validate.shape[0]}')
    
    return train, test, validate

## Scale